<a href="https://colab.research.google.com/github/Rish-P/Tensorflow-ConvNets_Rock-paper-scissors/blob/master/Rock_Paper_Scissors_Tensorflow_and_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow tensorflow_datasets

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import tensorflow_datasets as tfds

In [ ]:
import keras_tuner as kt

In [ ]:
pip install --upgrade keras-tuner

In [ ]:
builder = tfds.builder('rock_paper_scissors')
#builder.info

In [ ]:
ds_train = tfds.load('rock_paper_scissors',split='train')
ds_test = tfds.load('rock_paper_scissors',split='test')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incomplete6D00MT/rock_paper_scissors-train.tfrecord


  0%|          | 0/2520 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incomplete6D00MT/rock_paper_scissors-test.tfrecord


  0%|          | 0/372 [00:00<?, ? examples/s]

Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


## CHECKING THE ITEMS IN DATASET

In [ ]:
for item in ds_train:
  print(item['image'].shape)
  print(item['label'])
  break


## PREPARING IMAGE DATA INTO NUMPY ARRAY

In [ ]:
# use list comprehension to convert both dataframes into numpy arrays
train_X = np.array([item['image'].numpy()[:,:,1] for item in ds_train])
train_y = np.array([item['label'].numpy() for item in ds_train])

test_X = np.array([item['image'].numpy()[:,:,1] for item in ds_test])
test_y = np.array([item['label'].numpy() for item in ds_test])

###reshape the images,
###convert to float values,
###normalize the pixel values

In [ ]:
train_y.shape

In [ ]:
train_X = train_X.reshape(2520, 300,300,1)
test_X = test_X.reshape(372, 300,300,1)

train_X.astype('float32')
test_X.astype('float32')

train_X = train_X / 255
test_X = test_X / 255

In [ ]:
train_X.shape

In [ ]:
model = keras.Sequential([
                          # we are making a DNN similar to what LeNet had
       keras.layers.Conv2D(64,(3,3),input_shape=(300,300,1)),  
       keras.layers.MaxPool2D(pool_size=(2,2)),           
       keras.layers.Activation(activation='relu'),

       keras.layers.Conv2D(64,(3,3)),  
       keras.layers.MaxPool2D(pool_size=(2,2)),           
       keras.layers.Activation(activation='relu'),

       keras.layers.Flatten(),
       keras.layers.Dense(128, activation='relu'),

       keras.layers.Dense(3, activation='sigmoid')
             
])


In [ ]:
print(model.summary())

In [ ]:
train_y.shape

In [ ]:
model.compile(
    optimizer='adam',
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
model.fit(train_X, train_y, epochs=3,batch_size=32)

## TRAIN A BETTER CONV2D, WITH TUNING

In [ ]:
def model_builder(hp):
  inputs = keras.Input(shape=(300, 300, 1))
  model = keras.Sequential()
  x = inputs
  for i in range(hp.Int("cnn_layers", 1, 3)):
    x = keras.layers.Conv2D(
    hp.Int(f"filters_{i}", 32, 128, step=32),
    kernel_size=(3, 3),
    activation="relu",
    )(x)
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
  x = keras.layers.Flatten()(x)

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))

  x = keras.layers.Dropout(0.5)(x)

    # The last layer contains 10 units,
    # which is the same as the number of classes.
  outputs = keras.layers.Dense(units=3, activation="softmax")(x)
  model = keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model.
  model.compile(
        loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam",
  )

  return model

In [ ]:
hp = kt.HyperParameters()


model = model_builder(hp)



In [ ]:
tuner = kt.RandomSearch(
    model_builder,
    max_trials=10,
    objective="val_accuracy",
)

In [ ]:
tuner.search(
    train_X,
    train_y,
    validation_split=0.2,
    epochs=3,
)

Trial 10 Complete [00h 00m 42s]
val_accuracy: 0.9980158805847168

Best val_accuracy So Far: 0.9980158805847168
Total elapsed time: 00h 10m 41s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.search_space_summary()


Search space summary
Default search space size: 5
cnn_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
filters_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}


In [ ]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]

In [ ]:
best_model.build(input_shape=(300, 300, 1))

In [ ]:
best_model.fit(train_X, train_y,batch_size=32,epochs = 5)

In [ ]:
best_model.evaluate(test_X,test_y)

12/12 [==============================] - 1s 65ms/step - loss: 0.9070 - accuracy: 0.7177


[0.9069793820381165, 0.7177419066429138]